In [1]:
%pip install deepeval

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
from deepeval import evaluate
import os
from dotenv import load_dotenv
from deepeval.metrics import GEval, FaithfulnessMetric, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase, LLMTestCaseParams

In [3]:
def load_data_from_json(file_path):
    """Loads data from a single JSON file."""
    with open(file_path, "r") as f:
        data = json.load(f)
    return data

def load_data_from_multiple_files(directory_path, file_extension="json"):
    """Loads data from multiple files (either JSON or JSONL)."""
    data = []
    for filename in os.listdir(directory_path):
        if filename.endswith(file_extension):
            file_path = os.path.join(directory_path, filename)
            if file_extension == "json":
                data.append(load_data_from_json(file_path))
    return data

In [4]:
directory_path = "qa_results_simple_retriever"
file_extension = "json"

data = load_data_from_multiple_files(directory_path, file_extension=file_extension)

In [5]:
questions = []
gt_answers = []
generated_answers = []
retrieved_documents = []
for item in data:
    questions.append(item["question"])
    gt_answers.append(item["ground_truth_answer"])
    generated_answers.append(item["generated_answer"])
    retrieved_documents.append(item["retrieved_documents"])

In [6]:
def create_deep_eval_test_cases(questions, gt_answers, generated_answers, retrieved_documents):
    return [
        LLMTestCase(
            input=question,
            expected_output=gt_answer,
            actual_output=generated_answer,
            retrieval_context=retrieved_document
        )
        for question, gt_answer, generated_answer, retrieved_document in zip(
            questions, gt_answers, generated_answers, retrieved_documents
        )
    ]

In [7]:
test_cases = create_deep_eval_test_cases(questions, gt_answers, generated_answers, retrieved_documents)

In [8]:
from deepeval.metrics import (ContextualPrecisionMetric, ContextualRecallMetric, ContextualRelevancyMetric)
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric

In [9]:
contextual_precision_scores = []
contextual_recall_scores = []
contextual_relevancy_scores = []
answer_relevancy_scores = []
faithfulness_scores = []
# Run it for the first 50 test cases
for i, test_case in enumerate(test_cases[:50]):
    print(f"Processing test case {i+1}/{len(test_cases)}")
    # retrieval metrics
    contextual_precision = ContextualPrecisionMetric()
    contextual_recall = ContextualRecallMetric()
    contextual_relevancy = ContextualRelevancyMetric()
    # generation metrics
    answer_relevancy = AnswerRelevancyMetric()
    faithfulness = FaithfulnessMetric()
    # evaluate
    contextual_precision.measure(test_case)
    print("Computed contextual precision")
    contextual_recall.measure(test_case)
    print("Computed contextual recall")
    contextual_relevancy.measure(test_case)
    print("Computed contextual relevancy")
    answer_relevancy.measure(test_case)
    print("Computed answer relevancy")
    faithfulness.measure(test_case)
    print("Computed faithfulness")
    # store results
    contextual_precision_scores.append(contextual_precision.score)
    contextual_recall_scores.append(contextual_recall.score)
    contextual_relevancy_scores.append(contextual_relevancy.score)
    answer_relevancy_scores.append(answer_relevancy.score)
    faithfulness_scores.append(faithfulness.score)

Processing test case 1/192


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

...

Computed contextual relevancy


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Computed answer relevancy


Computed faithfulness


In [10]:
contextual_precision_scores, contextual_recall_scores, contextual_relevancy_scores, answer_relevancy_scores, faithfulness_scores

([0.5833333333333333,
  0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0,
  1.0,
  1.0,
  1.0,
  0.8333333333333333,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.5833333333333333,
  1.0,
  1.0,
  1.0,
  0.8333333333333333,
  0.8333333333333333,
  1.0,
  1.0,
  1.0,
  0.5833333333333333,
  0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0,
  1.0,
  0,
  0,
  0.5,
  0.8333333333333333,
  1.0,
  0.3333333333333333,
  0.8333333333333333,
  1.0,
  0.5833333333333333,
  1.0,
  0.8333333333333333,
  1.0,
  1.0],
 [0.6428571428571429,
  0.8461538461538461,
  0.38461538461538464,
  0.7647058823529411,
  0.4090909090909091,
  0.5384615384615384,
  0.7142857142857143,
  0.8235294117647058,
  0.625,
  0.45454545454545453,
  0.7727272727272727,
  0.5714285714285714,
  0.95,
  0.75,
  0.9565217391304348,
  0.75,
  0.9230769230769231,
  0.375,
  0.2631578947368421,
  0.5263157894736842,
  0.8,
  0.5,
  0.8666666666666667,
  0.5,
  0.9,
  0.5294117647058824,
  0.6666666666666666,
  0.5882352941176471,
  0.55

In [12]:
import pandas as pd

# Create DataFrame
df_scores = pd.DataFrame({
    "Contextual Precision": contextual_precision_scores,
    "Contextual Recall": contextual_recall_scores,
    "Contextual Relevancy": contextual_relevancy_scores,
    "Answer Relevancy": answer_relevancy_scores,
    "Faithfulness": faithfulness_scores
})

df_scores

,Contextual Precision,Contextual Recall,Contextual Relevancy,Answer Relevancy,Faithfulness
0,0.583333,0.642857,0.633333,1.000000,0.916667
1,0.000000,0.846154,0.555556,0.931034,0.923077
2,1.000000,0.384615,0.730769,1.000000,0.928571
3,1.000000,0.764706,0.756757,1.000000,0.875000
4,1.000000,0.409091,0.642857,1.000000,1.000000
5,1.000000,0.538462,0.836735,0.952381,1.000000
6,1.000000,0.714286,0.529412,0.823529,1.000000
7,1.000000,0.823529,0.739130,1.000000,1.000000
8,1.000000,0.625000,0.585366,1.000000,1.000000
9,0.000000,0.454545,0.636364,0.893617,0.916667


In [13]:
# compute average scores
average_scores = df_scores.mean()
average_scores

Contextual Precision    0.803333
Contextual Recall       0.643244
Contextual Relevancy    0.677705
Answer Relevancy        0.891988
Faithfulness            0.961027
dtype: float64